## EpergyPlus + Python + Eppy
This script is an exercise for the workshop<br>
Prepared by: Pedram Nojedehi <br>
Created on: 18/07/2025

In [43]:
from eppy.modeleditor import IDF
import os

idd_path = r"C:\EnergyPlusV24-1-0\Energy+.idd"
idf_path = r"Shoebox.idf"
epw_path = r"ON_OTTAWA-INTL-ONT_716280_19.epw"

output_dir = "window_scenarios"
os.makedirs(output_dir, exist_ok=True)

IDF.setiddname(idd_path)
idf = IDF(idf_path, epw_path)

In [44]:
idf.idfobjects['FenestrationSurface:Detailed'][0].Construction_Name

'Exterior Window'

In [45]:
from eppy.modeleditor import IDF
import os

# Set IDD file once
IDF.setiddname(idd_path)

# Define scenarios
scenarios = {
    "lowSHGC": {"U": 1.4, "SHGC": 0.3},
    "medSHGC": {"U": 1.4, "SHGC": 0.5},
    "highSHGC": {"U": 1.4, "SHGC": 0.7},
    "lowU": {"U": 1.0, "SHGC": 0.5},
    "highU": {"U": 2.5, "SHGC": 0.5}
}

original_window_construction = "Exterior Window"

for scenario_name, props in scenarios.items():
    # Load a fresh copy of the base model for each scenario
    idf_scenario = IDF(idf_path, epw_path)

    # Create new glazing definition
    glazing_name = f"Glazing_{scenario_name}"
    idf_scenario.newidfobject(
        "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
        Name=glazing_name,
        UFactor=props["U"],
        Solar_Heat_Gain_Coefficient=props["SHGC"],
        Visible_Transmittance=0.6
    )

    # Create new construction using the glazing
    new_construction_name = f"Exterior Window {scenario_name}"
    idf_scenario.newidfobject(
        "CONSTRUCTION",
        Name=new_construction_name,
        Outside_Layer=glazing_name
    )

    # Assign the new construction to all windows using the original one
    for win in idf_scenario.idfobjects["FENESTRATIONSURFACE:DETAILED"]:
        if win.Construction_Name == original_window_construction:
            win.Construction_Name = new_construction_name

    # Save the scenario-specific IDF
    scenario_file = os.path.join(output_dir, f"model_window_{scenario_name}.idf")
    idf_scenario.saveas(scenario_file)

In [46]:
idf.idfobjects['FenestrationSurface:Detailed'][0].Construction_Name

'Exterior Window'

In [47]:
for file in os.listdir(output_dir):
    if file.endswith(".idf"):
        run_dir = os.path.join(output_dir, file.replace(".idf", ""))
        os.makedirs(run_dir, exist_ok=True)

        idf = IDF(os.path.join(output_dir, file), epw_path)
        idf.run(output_directory=run_dir, expandobjects=True)



C:\EnergyPlusV24-1-0\energyplus.exe --weather G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\ON_OTTAWA-INTL-ONT_716280_19.epw --output-directory G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\window_scenarios\model_window_lowSHGC --idd C:\EnergyPlusV24-1-0\Energy+.idd --expandobjects G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\window_scenarios\model_window_lowSHGC_2b1ead.idf


C:\EnergyPlusV24-1-0\energyplus.exe --weather G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\ON_OTTAWA-INTL-ONT_716280_19.epw --output-directory G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\window_scenarios\model_window_medSHGC --idd C:\EnergyPlusV24-1-0\Energy+.idd --expandobjects G:\Other computers\My Laptop (1)\Dissemination\eppy_API_workshop\EnergyPlus-python-EMS-main\window_scenari

In [48]:
import pandas as pd
import os

results = []

for file in os.listdir(output_dir):
    if file.endswith(".idf") and file.startswith("model_window_"):
        scenario = file.replace("model_window_", "").replace(".idf", "")
        run_dir = os.path.join(output_dir, f"model_window_{scenario}")
        
        meter_path = os.path.join(run_dir, "eplusmtr.csv")
        out_path = os.path.join(run_dir, "eplusout.csv")

        if not os.path.exists(meter_path) or not os.path.exists(out_path):
            print(f"Missing output files for scenario: {scenario}")
            continue

        # Read meter data and convert J → kWh
        df_meter = pd.read_csv(meter_path)
        # print(df_meter)
        heating_j = df_meter["HeatingCoils:EnergyTransfer [J](Hourly)"].sum()
        cooling_j = df_meter["CoolingCoils:EnergyTransfer [J](Hourly)"].sum()
        total_energy_kwh = (heating_j + cooling_j) / 3.6e6

        # Read zone temperature and setpoints
        df_out = pd.read_csv(out_path)
        zone_temp = df_out["ZONE1:Zone Mean Air Temperature [C](Hourly)"]
        cooling_setpoint = df_out["ZONE1:Zone Thermostat Cooling Setpoint Temperature [C](Hourly)"]
        overheating_hours = (zone_temp > cooling_setpoint).sum()

        results.append({
            "Scenario": scenario,
            "Total_Energy_kWh": round(total_energy_kwh, 2),
            "Overheating_Hours": overheating_hours
        })

df_results = pd.DataFrame(results)
print(df_results)


   Scenario  Total_Energy_kWh  Overheating_Hours
0   lowSHGC           5859.79                723
1   medSHGC           6984.24                787
2  highSHGC           8147.37                828
3      lowU           7055.22                793
4     highU           6931.55                780
